In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the iexfinance api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [9]:
from iexfinance.stocks import get_historical_data
import iexfinance as iex

# Data Collection

In this step, you will need to use the IEX api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [10]:
list_of_tickers = ["SPY", "AGG"]
# YOUR CODE HERE
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for SPY and AGG
IEXCLOUD_df = get_historical_data(list_of_tickers, start_date, end_date, close_only=True, output_format='pandas')
#df = get_historical_data(list_of_tickers, start_date, end_date, close_only=True, output_format='pandas')
IEXCLOUD_df.head()

SPY                AGG         
             close     volume   close   volume
date                                          
2019-01-22  262.86  115531183  106.44  6823956
2019-01-23  263.41   86030287  106.55  3222846
2019-01-24  263.55   59204139  106.73  7308898
2019-01-25  265.78   96883358  106.68  3469453
2019-01-28  263.76   85613655  106.62  7365747

In [11]:
#Saving the data to a csv file created with the IEXCLOUD extract, to avoid multiple IEXCLOUD requests
IEXCLOUD_df.drop(columns=['volume'], level=1, inplace=True)
IEXCLOUD_df.columns=["SPY", "AGG"]
IEXCLOUD_df.to_csv("../Resources/IEXCLOUD_data.csv", encoding='utf-8')#, index=False)

In [51]:
#Getting the data from a csv file created with the IEXCLOUD extract, to avoid multiple IEXCLOUD requests
from pathlib import Path
IEXCLOUD_csv = Path("../Resources/IEXCLOUD_data.csv")
IEXCLOUD_df = pd.read_csv(IEXCLOUD_csv, index_col="date", parse_dates=True, infer_datetime_format=True)
IEXCLOUD_df.tail()

,SPY,AGG
date,,
2020-01-13,327.95,112.91
2020-01-14,327.45,113.01
2020-01-15,328.19,113.18
2020-01-16,330.92,113.18
2020-01-17,331.95,113.05


# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [34]:
# Calculate the daily roi for the stocks
# YOUR CODE HERE
daily_returns = IEXCLOUD_df.pct_change()
daily_returns.head()

,SPY,AGG
date,,
2019-01-22,NaN,NaN
2019-01-23,0.002092,0.001033
2019-01-24,0.000531,0.001689
2019-01-25,0.008461,-0.000468
2019-01-28,-0.007600,-0.000562


In [61]:
# Calculate volatility
# YOUR CODE HERE
avg_daily_return_SPY = daily_returns.mean()['SPY']
avg_daily_return_AGG = daily_returns.mean()['AGG']
print (f"Avg return SPY             Avg return AGG")
print (f"{avg_daily_return_SPY} ---- {avg_daily_return_AGG}")
std_daily_return_SPY = daily_returns.std()['SPY']
std_daily_return_AGG = daily_returns.std()['AGG']
print ("------------------------------------------------------")
print (f"STD daily returns SPY     STD daily returns AGG")
print (f"{std_daily_return_SPY} ---- {std_daily_return_AGG}")

Avg return SPY             Avg return AGG
0.0009612642291769476 ---- 0.00024318932345666555
------------------------------------------------------
STD daily returns SPY     STD daily returns AGG
0.007401574798510325 ---- 0.002084866830981002


In [52]:
# Save the last day's closing price
# YOUR CODE HERE
SPY_last_close = IEXCLOUD_df["SPY"][-1]
AGG_last_close = IEXCLOUD_df["AGG"][-1]
print (f"{SPY_last_close} ---- {AGG_last_close}")

331.95 ---- 113.05


In [69]:
# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30
monte_carlo = pd.DataFrame()

In [79]:
# Run the Monte Carlo Simulation
simulated_price_df = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

for x in range(number_simulations):
    # YOUR CODE HERE
    simulated_SPY_prices = [SPY_last_close]
    simulated_AGG_prices = [AGG_last_close]
    # Simulate the returns for 252 days
    for i in range(number_records):
        simulated_SPY_price = simulated_SPY_prices[-1] * (1 + np.random.normal(avg_daily_return_SPY, std_daily_return_SPY))
        simulated_AGG_price = simulated_AGG_prices[-1] * (1 + np.random.normal(avg_daily_return_AGG, std_daily_return_AGG))
        
        # Append the simulated price to the list
        simulated_SPY_prices.append(simulated_SPY_price)
        simulated_AGG_prices.append(simulated_AGG_price)
        
    # Append a simulated prices of each simulation to DataFrame
    simulated_price_df["SPY prices"] = pd.Series(simulated_SPY_prices)
    simulated_price_df["AGG prices"] = pd.Series(simulated_AGG_prices)
    
    # Set the portfolio weights (60% SPY; 40% AGG)
    weights = [0.60, 0.40]

    # Use the `dot` function with the weights to multiply weights with each column's simulated daily returns
    portfolio_daily_returns = simulated_price_df.dot(weights)
    
    # Calculate the normalized, cumulative return series
    monte_carlo[x] = portfolio_daily_returns
    
monte_carlo.head()

0    0         244.390000
1         244.788297
2   ...
1    0          244.390000
1          244.721308
2 ...
2    0          244.390000
1          246.408463
2 ...
3    0          244.390000
1          244.003722
2 ...
4    0          244.390000
1          244.389268
2 ...
dtype: object

In [77]:
portfolio_cumulative_returns.tail()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
7556,227463.455244,76019.018576,272907.179771,82767.590965,153580.618748,95569.534377,308772.568616,272661.497913,149190.852595,389110.071492,...,938883.027346,83814.808656,99180.169220,423559.939836,453147.659378,258710.529016,376035.780473,117618.889337,123895.937791,690728.860544
7557,226770.030639,75901.267717,272193.944754,83706.570210,153036.384918,95879.676626,310574.315450,271202.200180,148197.314973,391094.704697,...,947944.571342,83589.014201,100383.694762,428544.702517,447028.164747,259450.420368,373209.194696,117580.604866,122160.535610,693353.713355
7558,225298.430649,75951.101274,277352.962322,85144.116851,152200.109319,95825.188811,315308.121303,276132.120791,148543.891917,388022.420096,...,962358.637685,84055.818682,99110.632477,426088.412079,450949.782683,260303.836924,370088.299693,117100.704504,123485.160101,685908.689953
7559,225723.470274,76590.519217,275171.012647,84704.800428,152017.448641,95408.368231,318573.456009,274122.081397,149592.078237,389221.873499,...,965249.417234,85530.863304,99452.175224,421374.150695,455296.916716,259284.351820,369794.232359,119301.936368,124023.362167,688889.371767
7560,225989.063414,76806.618640,275784.371516,85441.501458,151686.054099,95772.177334,319932.916029,273283.222658,149990.138198,390771.082975,...,967329.878769,85046.386228,97957.529186,426392.350086,456471.993718,257534.673316,374880.891578,120199.883299,123766.242358,689806.622126


In [ ]:
# Visualize the Simulation
# YOUR CODE HERE

In [14]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
# YOUR CODE HERE

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)
# YOUR CODE HERE

In [ ]:
# Display the 90% confidence interval for the ending returns
# YOUR CODE HERE

In [ ]:
# Visualize the distribution of the ending returns
# YOUR CODE HERE

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# YOUR CODE HERE

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# YOUR CODE HERE

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE